In [1]:
import joblib
import pandas as pd

In [8]:

def recomendacion_juego( game_id : int ):
    # carga de archivos
    knn_load = joblib.load(r'./dataquery/modelo_knn.pkl')
    tfidf_matrix= joblib.load(r'./dataquery/tfidf_matrix.pkl')
    df = pd.read_csv(r'./dataquery/game_profile.csv')

    if game_id not in df['id'].values:
        return f"Game Id '{game_id}' not found"
    # trae el index donde se encuetra el game_id
    idx = df.index[df['id'] == game_id].tolist()[0]

    # funcion para traer el nombre del juego
    name = df['title'].loc[idx]
    # print(f'Porque te gusto "{name}" te recomendamos:')

    
    values , indices = knn_load.kneighbors(tfidf_matrix[idx])

    
    # Exclude the first (index 0) since it'll be the game itself
    game_indices = indices[0][1:]
    response = {'titulo_buscado':name,
                'game_id':game_id,
                'recomendados':df[['title','id']].loc[game_indices].to_dict(orient='records')}
    return response

In [9]:
recomendacion_juego(47810)

{'titulo_buscado': 'Dragon Age: Origins - Ultimate Edition',
 'game_id': 47810,
 'recomendados': [{'title': 'Dragon Age™: Origins Awakening', 'id': 47730.0},
  {'title': 'Dragon Age: Origins', 'id': 17450.0},
  {'title': 'Mass Effect', 'id': 17460.0},
  {'title': 'Jade Empire™: Special Edition', 'id': 7110.0},
  {'title': 'Mass Effect 2', 'id': 24980.0}]}

In [ ]:

df = pd.read_csv(r'./dataquery/game_profile.csv')

In [6]:
recomendacion_juego(282010.0)

KeyError: 282010.0